In [34]:
import pandas as pd

In [49]:
def formatting_data(file_path):
    f = open(file_path, 'rt')

    data = f.readlines()

    final_list = []
    #storing in a list instead of a df, it was taking too long
    #df = pd.DataFrame(columns=['movie_id', 'user_id', 'rating', 'rating_date'])

    for i, line in enumerate(data):
        if ':' in line:
            current_movie_id = int(line[:-2])
        elif ',' in line:
            tmp = line[:-1].split(',')
            final_list.append([current_movie_id, int(tmp[0]), int(tmp[1]), tmp[2]])
            #df2 = pd.DataFrame([[current_movie_id, int(tmp[0]), int(tmp[1]), tmp[2]]], columns=['movie_id', 'user_id', 'rating', 'rating_date'])
            #df = df.append(df2, ignore_index=True)
    
    return final_list

In [50]:
data = formatting_data('netflix-prize-data/combined_data_1.txt')
data += formatting_data('netflix-prize-data/combined_data_2.txt')
#there are a couple more files, should we use all the data?
len(data)

51031355

In [52]:
movie_titles = pd.read_csv('netflix-prize-data/movie_titles.csv', sep = ',', names = ['movie_id', 'year_released', 'title'], encoding='latin1')